In [2]:
# okay, first get an idea of how many tokens are there
# why it is being counted, figure out the max list you can send to API. 
# the max number of tokens you can send to the API is 8000

import pandas as pd 
import os 


folder_year = r"C:\Users\Roberto\Documents\GitHub Repositories\USPTO\data\fake_2005_folder"

df = pd.read_csv(os.path.join(folder_year, "dataframe.csv"))


In [3]:
df.head(3)

# abstract problem 

,publication_title,publication_num,publication_date,application_type,classifications,inventors,abstracts,descriptions,claims,publication_number,ipc_classifications,national_classifications,abstract,description
0,Anthurium andreanum plant named 'Anthbondem',NaN,20050106,utility,NaN,"[{'last_name': 'Dijk', 'first_name': 'Jan'}]",NaN,NaN,['1\n. A new and distinct\nAnthurium andreanum...,20050005336,['\n07\nG11C019/08\nA01H005/00\n'],['\nUS\nPLT365000\n'],\nA new and distinct cultivar of Anthurium and...,\n\nLATIN NAME OF THE GENUS AND SPECIES OF THE...
1,Novelty jeans,NaN,20050106,utility,NaN,"[{'last_name': 'Goldkind', 'first_name': 'Tina'}]",NaN,NaN,['1\n. A novelty jeans formed of suitable fabr...,20050000001,['\n07\nA41D001/06\n'],['\nUS\n002227000\n'],\nNovelty play jeans for children which includ...,\n\nBACKGROUND OF THE INVENTION \n The inventi...
2,Garment featuring means for temporarily attach...,NaN,20050106,utility,NaN,"[{'last_name': 'Levy', 'first_name': 'Philippe...",NaN,NaN,"[""1\n. A temporary means for attaching a cloth...",20050000002,['\n07\nA41D027/08\n'],['\nUS\n002244000\n'],"\nA garment, such as a skirt, culottes, dress,...",\n\nFIELD OF THE INVENTION \n This invention c...


In [4]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


for txt_col in ["abstract", "description", "claims"]:
    # count tokens
    df[f"{txt_col}_tokens"] = df[txt_col].fillna("").apply(lambda x: num_tokens_from_string(x, "text-embedding-3-small"))

    # count characters
    df[f"{txt_col}_characters"] = df[txt_col].fillna("").apply(lambda x: len(x))

    print(df[f"{txt_col}_tokens"].max())
    print(df[f"{txt_col}_characters"].max())



738
3034
530181
1191746
67972
214028


In [5]:
# get a sense of the distribution of tokens. i.e. the percentiles
for txt_col in ["abstract", "description", "claims"]:
    print(f"Percentiles for {txt_col}:")
    print(df[f"{txt_col}_tokens"].quantile([0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]))


total_tokens = df["abstract_tokens"].sum() + df["description_tokens"].sum() + df["claims_tokens"].sum()
print("Total number of tokens:", total_tokens)
# abstract, no problem
# description, 68877.19!! that means it is like 9 times the max number of tokens you can send to the API
# claims, on the edge... 8000 tokens.
    

# try VPN + gemini to get infinite tokens
# try CLIP finetuned to do images. Finetune CLIP in a good simple way. Just get the batche4s and ensure you are indeed getting better diagonals. Should not be too hard.


Percentiles for abstract:
0.10     57.00
0.25     89.00
0.50    127.00
0.75    167.00
0.90    205.00
0.95    236.00
0.99    318.07
Name: abstract_tokens, dtype: float64
Percentiles for description:
0.10     2389.30
0.25     3943.50
0.50     6614.00
0.75    11634.75
0.90    20822.40
0.95    30598.20
0.99    68877.19
Name: description_tokens, dtype: float64
Percentiles for claims:
0.10     371.00
0.25     691.00
0.50    1135.00
0.75    1803.50
0.90    2839.00
0.95    4003.05
0.99    8509.38
Name: claims_tokens, dtype: float64
Total number of tokens: 140167455


In [6]:
# IT SEEMS THAT THERE ARE MANY NaN VALUES.


non_string_elements = [element for element in df["description"] if not isinstance(element, str)]
print(len(non_string_elements))
print(set(non_string_elements))

df.shape

414
{nan}


(11394, 20)

In [21]:

"""
price gemini vs price GPT4
max tokens 


# gemini

Embeddings for Multimodal: Text 	Generate embeddings using text as an input 	Text 	Embeddings 	$0.0002 / 1k characters input
Embeddings for Multimodal: Image 	Generate embeddings using image as an input 	Image 	Embeddings 	$0.0001 / image input

gpt 4
Model	Usage
text-embedding-3-small	$0.02 / 1M tokens
text-embedding-3-large	$0.13 / 1M tokens
ada v2	$0.10 / 1M tokens

"""

# 1 token = 4 characters. quick estimation: 

# 0.0002 $ / 1k characters 
# 0.02 $ / 1M tokens


gemini_price_per_char = 0.0002 / 1000  # $0.2 per 1k characters
gpt4_price_per_1m_tokens = 0.02  # $0.02 per 1M tokens
gpt4_price_per_token = gpt4_price_per_1m_tokens / 1e6 

# 1 token = 4 characters
gemini_price_per_token = gemini_price_per_char * 4

print("Gemini price per 1M token:", 1e6 * gemini_price_per_token)
print("GPT4 price per 1M token:", 1e6 * gpt4_price_per_token)

# a bit more than 2 times more expensive... still might be doable 
# calculate average number of tokens per document

average_tokens_abstract = df["abstract_tokens"].mean()
average_tokens_description = df["description_tokens"].mean()
average_tokens_claims = df["claims_tokens"].mean()

total_average_tokens = average_tokens_abstract + average_tokens_description + average_tokens_claims
print("Average tokens per document:", total_average_tokens)

# price per document
price_per_document_gemini = total_average_tokens * gemini_price_per_token
price_per_document_gpt4 = total_average_tokens * gpt4_price_per_token

print("Price per document (Gemini):", price_per_document_gemini)
print("Price per document (GPT4):", price_per_document_gpt4)

# price per 1M documents 
price_per_1m_documents_gemini = 1e6 * price_per_document_gemini
price_per_1m_documents_gpt4 = 1e6 * price_per_document_gpt4

print("Price per 1M documents (Gemini):", price_per_1m_documents_gemini)
print("Price per 1M documents (GPT4):", price_per_1m_documents_gpt4)

# not so expensive... 

Gemini price per 1M token: 0.8
GPT4 price per 1M token: 0.02
Average tokens per document: 12301.865455502897
Price per document (Gemini): 0.009841492364402318
Price per document (GPT4): 0.0002460373091100579
Price per 1M documents (Gemini): 9841.492364402318
Price per 1M documents (GPT4): 246.03730911005792


In [18]:

# calculate how much it would cost you again

total_number_characters = 0

# for txt_col in ["abstract", "description", "claims"]:
for txt_col in ["abstract", "description", "claims"]:
    total_number_characters += df[f"{txt_col}_characters"].sum()

print("Total number of characters:", total_number_characters)

# price for all documents 
# 0.0002 / 1k characters

price_1k_char_gemini = 0.0002
total_price_gemini =  price_1k_char_gemini * total_number_characters / 1000 
print("Total price (Gemini):", total_price_gemini)

# price per document
price_per_document_gemini = total_price_gemini / df.shape[0]
print("Price per document (Gemini):", price_per_document_gemini)

price_per_1m_documents_gemini = 1e6 * price_per_document_gemini
print("Price per 1M documents (Gemini):", price_per_1m_documents_gemini) # 10k 

# 10.600 total price for 1M documents PLUS IMAGES!!
# check if you have problems with MAX imput size with gemini!! yeah, it seems to be MAX at 2048...

# maybe have the code ready. By the time you come back, it will be cheaper with larger imput... 

# also, maybe try both? and see which one is better.
# no problem. Focus on GPT4... Start to create embeddings and do regressions

Total number of characters: 604922783
Total price (Gemini): 120.9845566
Price per document (Gemini): 0.01061826896612252
Price per 1M documents (Gemini): 10618.268966122521


In [10]:
# test the limits. Try 3 sentences of 8000 tokens ans see what you got
# try 2 sentences together and separately and check that the vector embeddings are the same or different (i.e. they read into each other )


from openai import OpenAI

# read key from a txt file
OPENAI_API_KEY = open("data/openAI_key.txt", "r").read()

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {OPENAI_API_KEY}"
}

client = OpenAI(api_key=OPENAI_API_KEY)


def get_embeddings(text: str, model: str) -> dict:
    """
    Get the embeddings of a given text using a given model.

    Args:
        text (str): The text to obtain the embeddings from.
        model (str): The model to use to obtain the embeddings.

    Returns:
        dict: The embeddings of the given text.
    """
    response = client.embeddings.create(
        input=text,
        model=model
    )

    return response.data

model="text-embedding-3-small"

In [31]:
sentence1 = "This is a test"
sentence2 =  "second text"

# output_1_sentences = get_embeddings(sentence1, model) 
# output_2_sentences = get_embeddings([sentence1, sentence2], model) 


# output_10k_sentences = get_embeddings(" ".join(["This is a test"] * 10000), model) # every request 



In [26]:
len(output_1_sentences)
len(output_2_sentences)
output_1_sentences[0].embedding == output_2_sentences[0].embedding


True

In [24]:
print(output_2_sentences[0].embedding )

[0.00988506618887186, -0.005540902726352215, 0.0068014683201909065, -0.03810417652130127, -0.018254263326525688, -0.041231658309698105, -0.007651153020560741, 0.03220026567578316, 0.01892443746328354, 0.00010708322952268645, 0.05894339829683304, 0.012478001415729523, -0.023376306518912315, -0.01909995824098587, 0.02549852430820465, 0.029583394527435303, -0.08131444454193115, 0.0017113371286541224, -0.020041393116116524, 0.02832282893359661, 0.02631230652332306, -0.008736196905374527, 0.03954026475548744, 0.0029479677323251963, 0.027365438640117645, -0.042635831981897354, -0.02433369867503643, 0.002638810081407428, 0.022307218983769417, -0.055337224155664444, 0.046720702201128006, -0.03437035158276558, -0.0306524820625782, -0.012837023474276066, -0.01659478433430195, -0.00032935262424871325, -0.00602358765900135, 0.03462565690279007, -0.03357252478599548, 0.0007743900059722364, -0.036061741411685944, -0.052528876811265945, 0.005177891813218594, -0.0045715440064668655, 0.0187808275222778

In [32]:
from call_GPT4_Chat_in_parallel import get_embedding_list
import asyncio

content_list = ["hello world!"] * 5
embeddings = asyncio.run(get_embedding_list(content_list, max_parallel_calls=500))
# check that last embedding is there
print(embeddings[-1][:10])

RuntimeError: asyncio.run() cannot be called from a running event loop

In [33]:
from call_GPT4_Chat_in_parallel import get_embedding_list
import asyncio

content_list = ["hello world!"] * 5000

# Function to get embeddings using the existing event loop
async def get_embeddings(content_list, batch_size):
    return await get_embedding_list(content_list, batch_size)

# Use the existing event loop instead of asyncio.run()
loop = asyncio.get_event_loop()
if loop.is_running():
    embeddings = await get_embeddings(content_list, 1)
else:
    embeddings = loop.run_until_complete(get_embeddings(content_list, 1))


In [5]:
import time 
text_date = time.strftime("%Y_%m_%d__%H_%M_%S", time.localtime())
text_date

'2024_03_13__15_06_14'

In [8]:
import pandas as pd

# Create a dataframe
data = {'Variable Name': ['text_date', 'time'],
        'Variable Type': ['str', 'module'],
        'Variable Value': ['2024_03_13__15_06_14', '<module \'time\' (built-in)>']}

toy_df = pd.DataFrame(data)

# Add a new column
toy_df['New Column'] = [[1], [2]]

# Print the dataframe
print(toy_df)


new_embeddings = [11, 22]

for i, val in enumerate(new_embeddings):
    toy_df["New Column"][i].append(val)

print(toy_df)


  Variable Name Variable Type              Variable Value New Column
0     text_date           str        2024_03_13__15_06_14        [1]
1          time        module  <module 'time' (built-in)>        [2]


In [ ]:
zip([[1], [2]])